# Técnicas e algoritmos de ataques ao RSA

# Método de Fatoração de Fermat

O método de fatoração de Fermat é baseado na ideia de que qualquer número \( n \) ímpar pode ser escrito como a diferença de dois quadrados:

$n = a^2 - b^2 = (a + b)(a - b).$

Com isso, o objetivo do método é encontrar \( a \) e \( b \) tais que a fatoração seja válida. 


Método de Fermat é mais eficiente quando os dois fatores \( p \) e \( q \) são próximos, ou seja, \( |p - q| \) é pequeno. Caso contrário, o algoritmo pode demorar muito, especialmente para números grandes como o fornecido.

In [5]:
import time

def isqrt(n):
    """
    Calcula a raiz quadrada inteira de n usando o método de Newton.
    
    Parâmetros:
    n (int): O número cujo valor inteiro da raiz quadrada será calculado.

    Retorna:
    int: A raiz quadrada inteira de n.
    """
    x = n
    y = (x + n // x) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

def fermat_factorization(n):
    """
    Realiza a fatoração de Fermat em um número composto n.
    
    Assumimos que n é um produto de dois primos ímpares próximos, p e q, 
    tal que n = p * q e p ≈ q.

    Parâmetros:
    n (int): O número a ser fatorado.

    Retorna:
    tuple: Os fatores primos p e q de n.

    Lança:
    AssertionError: Se n não for igual ao produto dos fatores encontrados.
    """
    a = isqrt(n)  # Estimativa inicial de a ≈ √n
    b2 = a * a - n
    b = isqrt(b2)
    iterations = 0

    # Iterar até encontrar valores válidos para a e b
    while b * b != b2:
        a += 1
        b2 = a * a - n
        b = isqrt(b2)
        iterations += 1

    p = a + b
    q = a - b

    assert n == p * q, "Erro: A fatoração não produziu o número original."

    print(f"Fatoração concluída em {iterations} iterações.")
    return p, q


if __name__ == "__main__":
    N = 60765409007  # número a ser fatorado
    start_time = time.time()  
    p, q = fermat_factorization(N)
    elapsed_time = time.time() - start_time  # Tempo decorrido
    print(f"Os fatores de {N} são:\n p = {p}\n q = {q}")
    print(f"Tempo necessário: {elapsed_time:.6f} segundos.")


Fatoração concluída em 28996270 iterações.
Os fatores de 60765409007 são:
 p = 58484513
 q = 1039
Tempo necessário: 217.724241 segundos.


# Realizando o ataque com o expoente $e$ pequeno

In [3]:
%pip install pycryptodome
%pip install gmpy2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from Crypto.Util.number import getPrime
from gmpy2 import iroot

def small_e_attack(c: int, e: int) -> int:
    """
    Realiza o ataque de expoente pequeno (small e attack) ao módulo RSA.
    Este ataque é baseado no fato de que, para um expoente pequeno e (e.g., e = 3),
    se a mensagem m elevada a e for menor que o módulo n, a cifra c pode ser
    revertida diretamente extraindo a raiz e-ésima de c.

    Parâmetros:
    c (int): O valor cifrado (ciphertext).
    e (int): O expoente público do RSA.

    Retorna:
    int: A mensagem decifrada.
    """
    return int(iroot(c, e)[0])

def main():
    # Gerar dois números primos grandes
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q  # Calcular o módulo RSA

    # Parâmetros RSA
    e = 3  # Expoente pequeno
    message = b"Realizando o ataque com o expoente e pequeno"  # Mensagem a ser cifrada,  <class 'bytes'> : bytes can only contain ASCII literal characters

    # Converter a mensagem para um número inteiro
    m = int.from_bytes(message, "big")

    # Garantir que a condição para o ataque seja satisfeita (m^e < n)
    assert pow(m, e) < n, "A mensagem elevada a e deve ser menor que o módulo n."

    # Cifrar a mensagem
    c = pow(m, e, n)

    # Testar o ataque
    decrypted = small_e_attack(c, e)

    # Converter a mensagem decifrada de volta para bytes
    decrypted_message = decrypted.to_bytes((decrypted.bit_length() + 7) // 8, "big")

    print(f"Mensagem cifrada: {c}")
    print(f"Mensagem decifrada: {decrypted_message.decode()}")

if __name__ == "__main__": # Executar o programa
    main()


Mensagem cifrada: 25743891341151752174023774875992421537711484283679503238501693880243033585371998196392965004800450501732431150390591749970057742922442681793184160130301634875420170473673917678524349908346774914024937015560788227262326451575093608413693682353702706506817456631108426403938929009654126651524051981028949170280735074383
Mensagem decifrada: Realizando o ataque com o expoente e pequeno


# Método de Pollard's p-1

In [2]:
import os
import sys
import math
from binascii import hexlify
from gmpy2 import (
    mpz, random_state, mpz_urandomb, next_prime, is_prime, gcd, lcm, powmod
)

if sys.version_info < (3, 9):
    math.gcd = gcd
    math.lcm = lcm

# Configuração de debug
_DEBUG = False

# Configuração da flag e do estado inicial do gerador aleatório
FLAG = "Método de Pollard's p-1!!!"
FLAG = mpz(hexlify(FLAG.encode()), 16)  # Convertendo a flag para um inteiro
SEED = mpz(hexlify(os.urandom(32)).decode(), 16)
STATE = random_state(SEED)

def get_prime(state, bits):
    """
    Gera um número primo com o número de bits especificado.

    Parâmetros:
    - state: Estado do gerador aleatório.
    - bits (int): Quantidade de bits do primo.

    Retorna:
    - mpz: Um número primo.
    """
    return next_prime(mpz_urandomb(state, bits) | (1 << (bits - 1)))

def get_smooth_prime(state, bits, smoothness=16):
    """
    Gera um primo "smooth", ou seja, um número primo que é próximo de
    um produto de fatores pequenos.

    Parâmetros:
    - state: Estado do gerador aleatório.
    - bits (int): Quantidade de bits desejada para o primo.
    - smoothness (int): Número de bits dos fatores pequenos.

    Retorna:
    - tuple: O primo gerado e seus fatores.
    """
    p = mpz(2)
    p_factors = [p]

    # Construir produto inicial de fatores pequenos
    while p.bit_length() < bits - 2 * smoothness:
        factor = get_prime(state, smoothness)
        p_factors.append(factor)
        p *= factor

    bitcnt = (bits - p.bit_length()) // 2

    # Ajustar até atingir o número desejado de bits
    while True:
        prime1 = get_prime(state, bitcnt)
        prime2 = get_prime(state, bitcnt)
        candidate = p * prime1 * prime2
        if candidate.bit_length() < bits:
            bitcnt += 1
            continue
        if candidate.bit_length() > bits:
            bitcnt -= 1
            continue
        if is_prime(candidate + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = candidate + 1
            break

    p_factors.sort()
    return p, p_factors

# Parâmetros RSA
e = 0x10001


# Gerar os primos p e q com fatores smooth
while True:
    p, p_factors = get_smooth_prime(STATE, 1024, 16)
    if len(p_factors) != len(set(p_factors)):
        continue

    q, q_factors = get_smooth_prime(STATE, 1024, 17)
    if len(q_factors) != len(set(q_factors)):
        continue

    factors = p_factors + q_factors
    if e not in factors:
        break

# Calcular os valores de RSA
n = int(p * q)
m = math.lcm(p - 1, q - 1)
d = pow(e, -1, m)

# Cifrar a mensagem
c = pow(int(FLAG), e, n)

In [3]:
from gmpy2 import fac
from Crypto.Util.number import GCD, long_to_bytes

def pollards_p_minus_1(n, e, c, a=2, B=65535):
    """
    Realiza a fatoração de um número composto `n` usando o método de Pollard's p-1.
    Depois utiliza os fatores para decifrar uma mensagem RSA.

    Parâmetros:
    - n (int): O módulo RSA a ser fatorado.
    - e (int): O expoente público do RSA.
    - c (int): A mensagem cifrada (ciphertext).
    - a (int): A base inicial usada no método de Pollard. Default é 2.
    - B (int): O limite inicial para o cálculo de b! (fatorial). Default é 65535.

    Retorna:
    - str: A mensagem decifrada.
    """
    while True:
        # Calcula b! (fatorial) e tenta encontrar um divisor comum com n
        b = fac(B)
        tmp2 = pow(a, b, n) - 1  # a^(b!) mod n - 1
        gcd_value = GCD(tmp2, n)

        # Avalia o valor do gcd e ajusta o limite B, se necessário
        if gcd_value == 1:
            B += 1  # Aumenta o limite B para continuar
        elif gcd_value == n:
            B -= 1  # Diminui o limite B para ajustar
        else:
            # Sucesso: Encontramos um fator p
            p = gcd_value
            q = n // p
            assert p * q == n, "Erro na fatoração: p * q != n"

            # Calcula o expoente privado d
            d = pow(e, -1, (p - 1) * (q - 1))

            # Decifra a mensagem
            m = pow(c, d, n)
            return long_to_bytes(m).decode()


# Executa o método de Pollard's p-1 e imprime a mensagem decifrada
if __name__ == "__main__":
    message = pollards_p_minus_1(n, e, c)
    print(f"Mensagem decifrada: {message}")


Mensagem decifrada: Método de Pollard's p-1!!!


In [6]:
import time
from math import gcd, isqrt
import emoji


class SimuladorAtaqueRSA:
    def __init__(self, n, e=None, c=None, m=None, d=None):
        self.n = n
        self.e = e
        self.c = c
        self.m = m
        self.d = d

    def fatoracao_fermat(self):
        print(f"\n🔍 Executando Fatoração de Fermat para N={self.n}")
        p, q = fermat_factorization(self.n)
        print(f"🎯 Os fatores primos de {self.n} são:\n p = {p}\n q = {q}")
        #print(luciano)

        return
       

    def pollard_p_menos_1(self, B=10**6):
        # print(f"\n⚡ Executando Método Pollard p-1")
        message = pollards_p_minus_1(n, e, c)
        print(f"Mensagem decifrada: {message}")
        

def menu_principal():
    print("""
  ██████╗ ███████╗ █████╗     █████╗ ████████╗████████╗ █████╗  ██████╗██╗  ██╗
  ██╔═██║ ██╔════╝██╔══██╗   ██╔══██╗╚══██╔══╝╚══██╔══╝██╔══██╗██╔════╝██║ ██║
  ██║ ██║ ██║     ██║  ██╗   ██║  ██╗   ██║      ██║   ██║  ██╗██║     ██║ ██║
  ████║   ███████╗███████║   ███████║   ██║      ██║   ███████║██║     ████║
  ██║ ██╗      ██║██╔══██║   ██╔══██║   ██║      ██║   ██╔══██║██║     ██╔═██║
  ██║  ██║███████║██║  ██║   ██║  ██║   ██║      ██║   ██║  ██║╚██████╗██║  ██║
  ╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝   ╚═╝  ╚═╝   ╚═╝      ╚═╝   ╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝

                                               .----.
                                .------------. | == |
                                |.-========-.| |----|
                                ||          || | == |
                                ||          || |----|
                                |'-..... ..-'| |::::|
                                  `"")---(""`  |___.|
                                /::::::::::::\"__  "
                               /::::=======:::\  \`\ 
              <!--    __________   __  ______  ______       -->
              <!--   / __/ ___| | / / / __/  |/  / _ | ___  -->
              <!--  / _// (_ /| |/ / / _// /|_/ / __ |/ _ \ -->
              <!-- /_/  \___/ |___/ /___/_/  /_/_/ |_/ .__/ -->
              <!--                                  /_/     -->  

12537508673247452740415065632494689029027732950928380086256250024867187926128380
80287907219466080126910572928733538110840068106336709993591705426997197501264265
37797307222843941099150274102793206236042838704774026115105372038660088467920614
59271374450331751970488423409043692985323238050322405992071593773700020204905132
60850861669417224690840035708894072063281961794947745531844449231173387949796417
48472790731929346064345414201887091548869282344794136434276044068819968354034453
81296531655214965307063040023445866227631412406489130525362013420587466334224983
906533723001807034527677883322082180058876687814462664149 %¨$$#¨&*(()((¨#@$&_)))
               

""")

    print("🤖 Bem-vindo ao Simulador de Ataques RSA!")
    escolha = int(input("""Escolha o tipo de ataque:\n
          [1] Fatoração de Fermat \n
          [2] Método de Pollard p-1\n
                        
          Sua escolha: """))
    



    t_inicio = time.perf_counter()
    if escolha == 1:
        print("🔑 Por favor, insira os dados necessários para o ataque:")
        n = int(input("🔢 Digite o número a ser fatorado: "))
        print("✅ Dados recebidos com sucesso!")
        ataque = SimuladorAtaqueRSA(n)
        resultado = ataque.fatoracao_fermat()
    elif escolha == 2:
        print(" Por favor, insira os dados necessários para o ataque:")
        n = int(input("🔑 Digite o valor de n (chave pública): ") )
        e = int(input("🔑 Digite o valor de e (chave pública): ") )
        c = int(input("🔐 Digite o valor de c (valor cifrado): "))
        print("✅ Dados recebidos com sucesso!")
        ataque = SimuladorAtaqueRSA(n,e,c)
        mensagens = [
        "[*] Hackeando sistemas...",
        "[*] Acesso autorizado!",
        "[*] Preparando ataque...",
        "[*] RSA Attack: Ativado!",
        "[*] Ataque completo!"
        ]
            
        time.sleep(1)

        print(f"\n⚡ Executando o Método Pollard p-1")

        for msg in mensagens:
            print(msg)
            time.sleep(2)  # Pausa entre as mensagens
        resultado = ataque.pollard_p_menos_1()

    else:
        print("⚠️ Escolha inválida.")
        return
    t_fim = time.perf_counter()

    # print("\n🎯 Resultado do ataque:")
    # print(f"{resultado}")
    print(f"⏱️ Tempo de execução: {t_fim - t_inicio:.2f} segundos")

if __name__ == "__main__":
    menu_principal()



  ██████╗ ███████╗ █████╗     █████╗ ████████╗████████╗ █████╗  ██████╗██╗  ██╗
  ██╔═██║ ██╔════╝██╔══██╗   ██╔══██╗╚══██╔══╝╚══██╔══╝██╔══██╗██╔════╝██║ ██║
  ██║ ██║ ██║     ██║  ██╗   ██║  ██╗   ██║      ██║   ██║  ██╗██║     ██║ ██║
  ████║   ███████╗███████║   ███████║   ██║      ██║   ███████║██║     ████║
  ██║ ██╗      ██║██╔══██║   ██╔══██║   ██║      ██║   ██╔══██║██║     ██╔═██║
  ██║  ██║███████║██║  ██║   ██║  ██║   ██║      ██║   ██║  ██║╚██████╗██║  ██║
  ╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝   ╚═╝  ╚═╝   ╚═╝      ╚═╝   ╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝

                                               .----.
                                .------------. | == |
                                |.-========-.| |----|
                                ||          || | == |
                                ||          || |----|
                                |'-..... ..-'| |::::|
                                  `"")---(""`  |___.|
                                /::::::::::::"__  "
              